In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../scripts/')
from lda_preprocessing import mark_peaks, retrieve_peak_dates

In [3]:
# get data
hashtag_df = pd.read_json('../../data/BTW17_Twitter/hashtags/hashtag_count.json')
peak_df = pd.read_json('../../data/BTW17_Twitter/peaks/peaks.json')

In [4]:
df = mark_peaks(hashtag_df, peak_df)
df.describe(include='all').T

  0%|          | 0/30595 [00:00<?, ?it/s]

In [6]:
output_df = retrieve_peak_dates(hashtag_df, df)

  0%|          | 0/30595 [00:00<?, ?it/s]

In [7]:
# save to json
path_file = '../../data/BTW17_Twitter/peaks/peak_dates.json'
output_df.to_json(path_file)